# RSA Decryption
- Ascii plaintext encoded using PKCS1.5

# PKCS1.5
```
RSA Modulo Size: e.g 2048 bits or 256 bytes
+------+------------------------------+------+--------------------+
| 0x02 | RANDOM NONZERO DIGITS        | 0x00 | MESSAGE IN ASCII   |
+------+------------------------------+------+--------------------+
```

In [1]:
# Given 

ct_string = "22096451867410381776306561134883418017410069787892831071731839143676135600120538004282329650473509424343946219751512256465839967942889460764542040581564748988013734864120452325229320176487916666402997509188729971690526083222067771600019329260870009579993724077458967773697817571267229951148662959627934791540"
E = 65537 

N_string = "179769313486231590772930519078902473361797697894230657273430081157732675805505620686985379449212982959585501387537164015710139858647833778606925583497541085196591615128057575940752635007475935288710823649949940771895617054361149474865046711015101563940680527540071584560878577663743040086340742855278549092581"
p_string = "13407807929942597099574024998205846127479365820592393377723561443721764030073662768891111614362326998675040546094339320838419523375986027530441562135724301"
q_string = "13407807929942597099574024998205846127479365820592393377723561443721764030073778560980348930557750569660049234002192590823085163940025485114449475265364281"

In [7]:
from os import urandom
from gmpy2 import mpz
from gmpy2 import invert, t_mod, mul, powmod

def decrypt(y, d, N):
    return powmod(y, d, N)


def encrypt(x, e, N):
    return powmod(x, e, N)


def decrypt_pipeline(c_decimal, d, N):
    m_decimal = decrypt(c_decimal, d, N)
    m_hex = hex(m_decimal)
    m = m_hex.split('00') #assumes correct format
    return  bytes.fromhex(m[1]).decode('utf8')


def encrypt_pipeline(message, e, N, randomhexstring=None):
    raw_message = bytes(message, 'utf8')
    TOTAL_LENGTH = 256
    APPENDLENGTH = TOTAL_LENGTH - len(raw_message) - 2
    
    if randomhexstring is None:
        r = urandom(APPENDLENGTH).hex()
    
    final_bytes = bytes.fromhex('02' + randomhex + '00') + raw_message
    final_decimal = mpz(int.from_bytes(final_bytes, 'big'))
    
    return encrypt(final_decimal, e, N)


N = mpz(N_string)
p = mpz(p_string)
q = mpz(q_string)
c = mpz(ct_string)
e = mpz(E)

# compute d 
phiN = N - p - q + 1
D = invert(e, phiN)
d = mpz(D)

# d * e mod phi(N) = 1 
# where phi(N) = N - p - q + 1
assert t_mod(mul(d, e), phiN)

In [8]:
print(decrypt_pipeline(mpz(ct_string), d, N))

Factoring lets us break RSA.


In [9]:
message = "Factoring lets us break RSA."
randomhex = '0805907610b524330594e51d5dbbf643f09603731e9817111392d0c64e2739959a092d4daf979d387520ea7e577af9eb50a29f736925e810ab2fb4640e091a0f73252cb669d5b62b26764190ed188239fe71e1a7cb9e935d2db55c98b024e1dae46d'

c_decimal = encrypt_pipeline(message, e, N, randomhex)

assert c == c_decimal

m = decrypt_pipeline(c_decimal, d, N)
print(m)

Factoring lets us break RSA.


In [10]:
message = "Factoring lets us break RSA."

c_decimal = encrypt_pipeline(message, e, N)

m = decrypt_pipeline(c_decimal, d, N)
print(m)

Factoring lets us break RSA.
